In [1317]:
#Dependencies
import os
import numpy as np
import pylab as pl
import tensorflow as tf
import matplotlib.pyplot as plt
from time import clock

import utils.movie_plotutils as plu
import utils.readMov as rmov
#import utils.dirutils as diru

#code to reload
import imp
imp.reload(plu)

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'dpi' : 200} #plotting pretty

#code to limit number of CPUs

#maxcpus = 2
#session_conf = tf.ConfigProto(
#     intra_op_parallelism_threads=maxcpus,
#     inter_op_parallelism_threads=maxcpus, allow_soft_placement=True, device_count = {'CPU': maxcpus})



In [1318]:
class aec_model(object):
    
    def __init__(self, params):
      params = self.add_params(params)
      self.params = params
      self.make_dirs()
      self.graph = self.make_graph()
    
    def add_params(self, params):  
        params['compression'] = params['imxlen']*params['imylen']/params['nneurons']
        params['savefolder'] = str('./output/movie_output/actfun_'+ params['model_type']+
                                   '_hiddenneurons_'+ str(params['nneurons'])+
                                   '_noisein_'+ str(params['noise_x'])+
                                   '_noiseout_'+ str(params['noise_r'])+
                                   '_lambda_'+ str(params['lambd'])+'/')
        return(params)
        
    def make_dirs(self):
        if not os.path.exists(self.params['savefolder']):
            os.makedirs(self.params['savefolder'])
        
    def make_graph(self):
    
        print('Compressing by',self.params['compression'],'for a total of',self.params['nneurons'],'neurons')

        #setup our graph
        #tf.reset_default_graph()
        mygraph = tf.Graph()
        with mygraph.as_default():
#CHANGE
            #input images
            with tf.name_scope('input'):
                self.x = tf.placeholder(tf.float32, shape=[self.params["batchsize"], 
                                                           self.params["time_patchsize"],self.params["imxlen"],self.params["imylen"], 1])

            #activation function type
            with tf.name_scope('activation_function'):
                self.act_fun = self.params['model_type']

            #noises
            with tf.name_scope('noises'):
                self.noisexsigma = self.params['noise_x']
                self.noisersigma = self.params['noise_r']

            #function to add noise
            with tf.name_scope("add_noise"):
                def add_noise(input_layer, std):
                    noise = tf.random_normal(shape=tf.shape(input_layer), mean=0.0, stddev=std, dtype=tf.float32) 
                    return tf.add(input_layer,noise)

            #weights
            with tf.variable_scope("weights"):
                
#CHANGE               
    
                weights_kernel = tf.random_normal([1,self.params['imxlen'],self.params['imylen'], 1,
                                                         self.params['nneurons']],
                                                        dtype=tf.float32,stddev=0.1)
                
                self.win = tf.get_variable('weights_in',initializer=weights_kernel)
                self.wout = tf.get_variable('weights_out',initializer=weights_kernel)
                #self.wout = tf.get_variable('weights_out',initializer=tf.transpose(weights_kernel))

            #bias
            with tf.variable_scope("bias"):
                self.bias = tf.Variable(tf.random_normal([self.params['nneurons']],dtype=tf.float32,stddev=0.1))

            #lambda
            with tf.name_scope('lambda'):
                self.lambd = self.params['lambd']

            #learning_rate
            with tf.name_scope('learning_rate'):
                self.learning_rate = self.params['learning_rate']

            #nonlienarities
            with tf.name_scope("nonlienarities"):
                #define nonlinearities
                def tanh_fun(arg):
                    return tf.nn.tanh(arg) 
                def sigmoid_fun(arg):
                    return tf.nn.sigmoid(arg) 
                def relu_fun(arg):
                    return tf.nn.relu(arg) 
                def no_fun(arg):
                    return arg

            #encoding part of model
            with tf.name_scope("encoding"):
                #calculate input
#CHANGE                
                linearin = tf.add(tf.nn.conv3d(add_noise(self.x,self.params['noise_x']), self.win, strides=[1,1,self.params['imxlen'],self.params['imylen'],1], padding='SAME'),self.bias) #Convolution over time, add noise to input, and multiply by weights
                #linearin = tf.add(tf.matmul(add_noise(self.x,self.params['noise_x']),self.win),self.bias) #add noise to input, and multiply by weights
                yin = tf.case({tf.equal(self.act_fun,'tanh'): (lambda: tanh_fun(linearin)),
                               tf.equal(self.act_fun,'sigmoid'): (lambda: sigmoid_fun(linearin)),
                               tf.equal(self.act_fun,'relu'): (lambda: relu_fun(linearin))},
                              default=(lambda: no_fun(linearin)),
                              exclusive=True)
                self.yin = add_noise(yin,self.params['noise_r'])
                #print(yin)
                self.yin = tf.case({tf.equal(self.act_fun,'tanh'): (lambda: tanh_fun(self.yin)),
                               tf.equal(self.act_fun,'sigmoid'): (lambda: sigmoid_fun(self.yin)),
                               tf.equal(self.act_fun,'relu'): (lambda: relu_fun(self.yin))},
                              default=(lambda: no_fun(self.yin)),
                              exclusive=True)

            #output part of model
            with tf.name_scope("decoding"):
                #calculate output (reconstruction)
#CHANGE                
                self.xp = tf.nn.conv3d_transpose(self.yin, self.wout, output_shape=(self.params["batchsize"], self.params["time_patchsize"],self.params["imxlen"],self.params["imylen"], 1), strides=[1,1,self.params['imxlen'],self.params['imylen'],1], padding='SAME') #Deconvolution 
                #self.xp = tf.nn.conv3d(self.yin, self.wout, output_shape=[self.params["batchsize"], self.params["time_patchsize"]*self.params["imxlen"]*self.params["imylen"],strides=[1,1,1,1,1], padding='SAME')
                
                #self.xp = tf.matmul(self.yin,self.wout) #add noise to inner layer, and multiply by weight transpose
                #self.xp = tf.case({tf.equal(self.act_fun,'tanh'): (lambda: tanh_fun(linearout)),
                #                    tf.equal(self.act_fun,'sigmoid'): (lambda: sigmoid_fun(linearout)),
                #                    tf.equal(self.act_fun,'relu'): (lambda: relu_fun(linearout))},
                #                    default=(lambda: no_fun(linearout)),
                #                    exclusive=True, name='output_nonlienarity')
             

            #calculate cost
            with tf.name_scope("cost_function"):
                self.cost = tf.sqrt(tf.reduce_mean(tf.square(self.x-self.xp))) - tf.reduce_sum(tf.abs(self.yin*self.lambd))

            #train our model
            with tf.name_scope("training_step"):
                self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cost)

            # create a summary for our cost, im, reconstruction, & weights
            with tf.name_scope('cost_viz'):
                tf.summary.scalar("cost", self.cost)

            #with tf.name_scope('image_viz'):    
            #    x_t = tf.reshape(self.x,(self.params['batchsize'], 0, self.params['imxlen'],self.params['imylen'],1))
            #    tf.summary.image("image", x_t, max_outputs=self.params["batchsize"])

            #with tf.name_scope('recon_viz'):
            #    xp_t = tf.reshape(self.xp,(self.params['batchsize'], 0,self.params['imxlen'],self.params['imylen'],1))
            #    print (xp_t)
            #    tf.summary.image("recon", xp_t,max_outputs=self.params["batchsize"])
#CHANGE
            with tf.name_scope('inweights_viz'):    
                #inwin_t = tf.reshape(tf.transpose(self.win),
                inwin_t = tf.reshape(tf.transpose(self.win, perm=[4,1,2,3,0]), (self.params['nneurons'], self.params['imxlen'], self.params['imylen'],1))
                tf.summary.image("inweights", inwin_t, max_outputs=self.params['nneurons'])
                
            with tf.name_scope('outweights_viz'):    
                outwin_t = tf.reshape(tf.transpose(self.wout, perm=[4,1,2,3,0]), (self.params['nneurons'], self.params['imxlen'], self.params['imylen'],1))
                tf.summary.image("outweights", outwin_t, max_outputs=self.params['nneurons'])
                      
            #with tf.name_scope('activnonlin_viz'):
            #    activation = tf.transpose(self.yin, perm=[0,4,1,2,3])
            #    activation = tf.reshape(activation, (self.params["batchsize"], self.params["time_patchsize"], -1))  #reshape nonlinear-vector [batchsize, nneurons, time] 
            #    activ_help_temp = activation[0,0] # temporarily take only one nneuron over time
            #    tf.summary.scalar("activnonlin", activ_help_temp)     
                
                
            # merge all summaries into a single "operation" which we can execute in a session 
            self.summary_op = tf.summary.merge_all()

        return(mygraph)

In [1319]:
#make session and train model
def train_model(aec):
    session_conf = tf.ConfigProto()
    session_conf.gpu_options.allow_growth = True
    #tf.device("/gpu:0")
    with tf.Session(graph = aec.graph, config = session_conf) as sess:   #config = session_conf

        #initialize vars
        init = tf.global_variables_initializer()
        sess.run(init)

        #summary writer for tensorboard
        writer = tf.summary.FileWriter(aec.params['savefolder'],
                                       graph=tf.get_default_graph())

        #save evolution of system over training
        cost_evolution = []
        wmean_evolution = []

        inweights_evolution = []
        outweights_evolution = []
        
        images = []
        recons = []
        print('{} hidden neurons, noise_in at {}, noise_out at {}, lambda at {}'.format(aec.params['nneurons'],
                                                                                          aec.params['noise_x'],
                                                                                          aec.params['noise_r'],
                                                                                          aec.params['lambd']))
        
        print('Training {} iterations in {} epochs... '.format(aec.params['iterations'],
                                                                   aec.params['epochs']))
        for epoch in range(aec.params['epochs']):
            start = clock()
            #print('Epoch {}: '.format(epoch+1))
            np.random.shuffle(m)
            #print ('Time_shuffle:', (clock()-start))
            for ii in range(aec.params['iterations']):
                
                #reshape our images for feeding to dict
                clip = np.reshape(m[ii*aec.params['batchsize']:(1+ii)*aec.params['batchsize'],:,:,:],(aec.params['batchsize'], aec.params["time_patchsize"], aec.params['imxlen'],aec.params['imylen'], 1)).astype(np.float32)

                #setup params to send to dictionary
                feeddict = {aec.x: clip}
    #                        aec.params['model_type']: params['model_type'],
    #                        aec.params['noise_x']: params['noise_x'],
    #                        aec.params['noise_r']: params['noise_r'],
    #                        aec.params['lambd']: params['lambd'],
    #                        aec.params['learning_rate']: params['learning_rate']
    #                        }

    

                #run our session
                sess.run(aec.train_step, feed_dict=feeddict)
                
                #yin = sess.run(aec.yin, feed_dict=feeddict)
                #print(yin.shape)

                #save evolution of params
                objcost, inws = sess.run([aec.cost, aec.win], feed_dict=feeddict)
                cost_evolution.append(objcost)
                wmean_evolution.append(np.mean(inws))

                #save detailed parameters 10 times over the total evolution
                if(((params['iterations']*epoch)+ii)%(int((aec.params['iterations']*aec.params['epochs'])/10))==0):
                    #dump our params
                    win, wout, img, recon = sess.run([aec.win, aec.wout, aec.x, aec.xp], feed_dict=feeddict)
                    #save our weights, image, and reconstruction
                    inweights_evolution.append(np.reshape(np.transpose(win, (1,2,4,3,0)), (params['imxlen'], params['imylen'], params['nneurons'],1)))
                    outweights_evolution.append(np.reshape(np.transpose(wout, (1,2,4,3,0)), (params['imxlen'], params['imylen'], params['nneurons'],1)))
                    #imshape = [aec.params['batchsize'],
                    #          aec.params['imxlen'],
                    #          aec.params['imylen']]
                    img_transformed = np.transpose(img, (1,4,0,2,3))
                    recons_transformed = np.transpose(recon, (1,4,0,2,3))
                    images.append(img_transformed[3][0])
                    recons.append(recons_transformed[3][0])

            end = clock()
            print ('Time needed for one epoch: ', "%.2f" % (end-start),'sec')
        #summarize final params
        summary, objcost, inws, outws = sess.run([aec.summary_op, aec.cost, aec.win, aec.wout], feed_dict=feeddict)
        cost_evolution.append(objcost)
        wmean_evolution.append(np.mean(inws))
        final_inweights = transform_weight_to_image(inws, aec.params)    #aec.win
        final_outweights = transform_weight_to_image(outws, aec.params)  #aec.wout
        #final_outweights = outws
        #print(tf.shape(final_outweights))
        writer.add_summary(summary,ii)
        writer.close()

    print('Done calculating!')
    
    return(cost_evolution,
            wmean_evolution,
            inweights_evolution,
            outweights_evolution,
            images,
            recons,
            final_inweights,
            final_outweights)

In [1320]:
#transform weight tensor to image format
def transform_weight_to_image(weight_vec, params):
    image = tf.transpose(weight_vec, perm=[1,2,4,0,3])
    image = tf.reshape(image, (params['imxlen'], params['imylen'], params['nneurons']))#,1))
    return image


In [1321]:
#set parameters for parameter sweep
params = {} #make a dictionary

#parameters constant for all
params["patchsize"] = 10
params["imxlen"] = params["patchsize"]
params["imylen"] = params["patchsize"]
params["time_patchsize"] = 10

params["iterations"] = 1000
params["epochs"] = 4

params["batchsize"] = 100
params["learning_rate"] = 0.01

#params for sweeping
nneurons = [25,64,100]

#lambdas = [0]
lambdas = [0,1e-6,1e-4, 1e-2]

#models = ['relu']
#models = ['tanh']
models = ['relu', 'sigmoid', 'tanh']


#batchsizes = [100,1000]
#learning_rates = [0.05,0.01,0.1]


noise_xs_rs_pairs = [[0.,0.], [1e-6,1e-4], [1e-4,1e-6], [1e-1,5e-3], [1e-3,5e-1]]
#noise_xs_rs_pairs = [[1e-6,1e-4], [1e-4,1e-6]]
#noise_xs_rs_pairs = [[0.,0.],
#noise_xs_rs_pairs = [[1e-7,1e-5],
#                    [1e-6,1e-4],
#                    [1e-5,1e-3],
#                    [1e-4,1e-2]]
#noise_xs = [1,1e-1,1e-3,1e-5,0]
#noise_rs = [1,1e-1,1e-3,1e-5,0]


In [1322]:
fpath = '/home/vasha/vanHaterenNaturalMovies/vid075'
fps = 25 #approximated from http://redwood.berkeley.edu/bruno/data/vid075/README and increased by me.
nframes = 9600
rawframeh = 128
rawframew = 128
barw = 16
framew = rawframew - barw #in pixels
frameh = rawframeh - barw #in pixels

#convert to degrees
ppd = 6 #pixels per degree subtended on retina (estimated 10deg for 64px in dong atick 95)
framewdeg = framew/ppd 
framehdeg = frameh/ppd
#sampling rate
deltawdeg = 1./ppd
deltahdeg = 1./ppd 
deltathz = 1./fps

In [1323]:
#vhimgs, params['nimages'] = imr.check_n_load_ims(params['patchsize'], params['iterations'])
m = rmov.readMov(fpath, nframes, rawframeh, rawframew, barw, patch_edge_size=params["patchsize"], time_size=params["time_patchsize"], normalize_patch=True)
m = np.transpose(m, (0, 3, 1, 2)) #change axis to [batchsize, time_patchsize, x_patchsize, y_patchsize]
np.random.shuffle(m)
np.shape(m)


(9600, 112, 112)
making patches...
(11, 9600, 112, 10)
(11, 11, 9600, 10, 10)
(960, 11, 11, 10, 10, 10)
(116160, 10, 10, 10)
normalizing patches...


(116160, 10, 10, 10)

In [ ]:
#load in movie
try:
    m
except NameError:
    print("Loading Movie...")
    m = rmov.readMov(fpath, nframes, rawframeh, rawframew, barw, patch_edge_size=params["patchsize"], time_size=params["time_patchsize"], normalize_patch=True)
    m = np.transpose(m, (0, 3, 1, 2)) #change axis to [batchsize, time_patchsize, x_patchsize, y_patchsize]
    np.random.shuffle(m)
    #mm = m - np.mean(m,axis=(1,2)).reshape(-1,1,1)
    
print("Movie Loaded. Shape is " + str(np.shape(m)))


#params of clips
imxlen = len(m[0,:,0,0])
imylen = len(m[0,0,:,0])
imflen = len(m[0,0,0,:])
nimages = len(m[:,0,0,0])

nimstrained = params["batchsize"] * params["iterations"]

if(nimstrained > nimages):
    print('ERROR! Trying to train',nimstrained,'patchess, but we only have',nimages,'patches!')
else:
    print('Training',nimstrained,'out of',nimages,'total patches.')

Movie Loaded. Shape is (116160, 10, 10, 10)
Training 100000 out of 116160 total patches.


In [ ]:
for neurons in nneurons:
    params['nneurons'] = neurons
    for model_type in models:
        params['model_type'] = model_type      
        for lambd in lambdas:
            params['lambd'] = lambd
            #for nx in noise_xs:i
                #params['noise_x'] = nx
                #for nr in noise_rs:
                    #params['noise_r'] = nr
            for xs,rs in noise_xs_rs_pairs:
                    params['noise_x'] = xs
                    params['noise_r'] = rs
                    #make our model
                    aec = aec_model(params)
                    #train it
                    [cost_evolution,
                     wmean_evolution,
                     inweights_evolution,
                     outweights_evolution,
                     images,
                     recons,
                     final_inweights,
                     final_outweights] = train_model(aec)

                    plu.save_plots(aec,
                                    cost_evolution,
                                    wmean_evolution,
                                    inweights_evolution,
                                    outweights_evolution,
                                    images,
                                    recons,
                                    final_inweights,
                                    final_outweights)
                    print('Done saving!')

Compressing by 4.0 for a total of 25 neurons
25 hidden neurons, noise_in at 0.0, noise_out at 0.0, lambda at 0
Training 1000 iterations in 4 epochs... 
Time needed for one epoch:  8.70 sec
Time needed for one epoch:  8.24 sec
Time needed for one epoch:  9.06 sec
Time needed for one epoch:  9.58 sec
Done calculating!
Done saving!
Compressing by 4.0 for a total of 25 neurons
25 hidden neurons, noise_in at 1e-06, noise_out at 0.0001, lambda at 0
Training 1000 iterations in 4 epochs... 
Time needed for one epoch:  9.50 sec
Time needed for one epoch:  8.56 sec
Time needed for one epoch:  8.38 sec
Time needed for one epoch:  8.62 sec
Done calculating!
Done saving!
Compressing by 4.0 for a total of 25 neurons
25 hidden neurons, noise_in at 0.0001, noise_out at 1e-06, lambda at 0
Training 1000 iterations in 4 epochs... 
Time needed for one epoch:  9.14 sec
Time needed for one epoch:  9.46 sec
Time needed for one epoch:  8.28 sec
Time needed for one epoch:  8.70 sec
Done calculating!
Done savin

In [ ]:
## show an example image
plt.imshow(m[274,1,:,:],cmap='gray',interpolation='none')
plt.colorbar()